In [1]:
from pybaseball import statcast_batter_expected_stats
from pybaseball import pitching_stats
from pybaseball import statcast_batter
from pybaseball import playerid_lookup
import pandas as pd
from scipy.stats import zscore
from pybaseball import cache

## Batters by wBOA ZScore

In [2]:
from pybaseball import batting_stats
# retrieve data on only players who have 50+ plate appearances this year
full_batting_2018 = batting_stats(2018, qual=150)
full_batting_2019 = batting_stats(2019, qual=150)
full_batting_2021 = batting_stats(2021, qual=150)
full_batting_2022 = batting_stats(2022, qual=150)
full_batting_2023 = batting_stats(2023, qual=150)

# Concatenate the DataFrames vertically
combined_batting_data = pd.concat([full_batting_2018, full_batting_2019,full_batting_2021, full_batting_2022, full_batting_2023], ignore_index=False)
full_batting_2022.head()

,IDfg


In [3]:
from pybaseball import statcast_batter_expected_stats

batting_18 = statcast_batter_expected_stats(2018, 150)
batting_19 = statcast_batter_expected_stats(2019, 150)
batting_21 = statcast_batter_expected_stats(2021, 150)
batting_22 = statcast_batter_expected_stats(2022, 150)
batting_23 = statcast_batter_expected_stats(2023, 150)

batting_data = pd.concat([batting_18, batting_19, batting_21, batting_22, batting_23], ignore_index=True)
batting_data

,last_name,first_name,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff
0,Lindor,Francisco,596019,2018,745,560,0.277,0.289,-0.012,0.519,0.512,0.007,0.368,0.375,-0.007
1,Turner,Trea,607208,2018,740,534,0.271,0.273,-0.002,0.416,0.415,0.001,0.331,0.335,-0.004
2,Machado,Manny,592518,2018,709,533,0.297,0.284,0.013,0.538,0.512,0.026,0.377,0.369,0.008
3,Hernández,César,514917,2018,708,454,0.253,0.242,0.011,0.362,0.352,0.010,0.320,0.316,0.004
4,Merrifield,Whit,593160,2018,707,526,0.304,0.281,0.023,0.438,0.428,0.010,0.349,0.343,0.006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629,Lopez,Nicky,670032,2023,215,153,0.235,0.230,0.005,0.317,0.295,0.022,0.298,0.290,0.008
1630,Madrigal,Nick,663611,2023,214,179,0.269,0.282,-0.013,0.363,0.346,0.017,0.307,0.309,-0.002
1631,Arroyo,Christian,624414,2023,206,153,0.241,0.231,0.010,0.369,0.341,0.028,0.274,0.269,0.005
1632,Espinal,Santiago,669289,2023,199,154,0.218,0.239,-0.021,0.296,0.319,-0.023,0.265,0.284,-0.019


In [4]:
# Filter the DataFrame for the relevant years (2019, 2021, 2022)
relevant_years = [2018, 2019, 2021, 2022]
filtered_batting_data = batting_data[batting_data['year'].isin(relevant_years)]


woba_2023 = batting_23[['player_id', 'woba']].rename(columns={'woba': 'woba_2023'})
grouped_data = filtered_batting_data .groupby(['player_id']).mean().reset_index().rename(columns={'woba': 'avg_woba'})

# Merge player names with the result DataFrame
# Merge player names and calculated results
players_batting_output = pd.merge(
    grouped_data,
    woba_2023,
    on=['player_id']
)

/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_3726/645265358.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_data = filtered_batting_data .groupby(['player_id']).mean().reset_index().rename(columns={'woba': 'avg_woba'})


In [22]:
column_order = [
    col for col in players_batting_output.columns if col not in ['avg_woba', 'woba_2023']
] + ['avg_woba', 'woba_2023']

players_batting_output = players_batting_output[column_order]

players_batting_output

,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,est_woba,est_woba_minus_woba_diff,zscore_difference,avg_woba,woba_2023
0,408234,2020.67,502.67,351.33,0.26,0.25,0.02,0.37,0.40,-0.03,0.31,-0.01,0.14,0.30,0.29
1,444482,2020.00,516.25,365.50,0.27,0.25,0.02,0.45,0.40,0.05,0.32,0.02,-0.67,0.33,0.30
2,453568,2020.00,622.25,457.50,0.28,0.28,0.01,0.48,0.45,0.03,0.34,0.01,0.28,0.35,0.36
3,455117,2020.00,395.75,252.50,0.20,0.19,0.00,0.35,0.33,0.01,0.27,0.00,-0.31,0.27,0.24
4,456781,2020.67,292.00,213.67,0.30,0.29,0.01,0.42,0.42,-0.01,0.33,-0.00,0.54,0.33,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,681082,2022.00,466.00,340.00,0.23,0.24,-0.00,0.36,0.34,0.02,0.28,0.01,1.58,0.29,0.34
240,682928,2022.00,302.00,238.00,0.25,0.24,0.01,0.32,0.32,0.01,0.27,0.00,1.56,0.27,0.31
241,682985,2022.00,418.00,259.00,0.25,0.25,0.01,0.36,0.39,-0.03,0.32,-0.01,1.86,0.30,0.36
242,683734,2021.50,512.00,370.00,0.25,0.25,0.00,0.41,0.42,-0.01,0.33,-0.01,0.40,0.32,0.32


In [ ]:
# print("Shape of the DataFrame:", combined_batting_data.shape)
# print("Column names:", combined_batting_data.columns.tolist())

In [ ]:
# # Filter the DataFrame for the relevant years (2019, 2021, 2022)
# relevant_years = [2019, 2021, 2022]
# filtered_data_batting = combined_batting_data[combined_batting_data['Season'].isin(relevant_years)]

# # Group the data by player and calculate the average wOBA for relevant years
# grouped_data = filtered_data_batting.groupby(['IDfg', 'Name']).mean().reset_index().rename(columns={'wOBA': 'avg_wOBA'})

# # Merge the average wOBA with the 2023 wOBA for each player
# woba_2023 = full_batting_2023[['IDfg', 'Name', 'wOBA']].rename(columns={'wOBA': 'wOBA_2023'})


# # Merge player names with the result DataFrame
# # Merge player names and calculated results
# players_off_output = pd.merge(
#     grouped_data,
#     woba_2023,
#     on=['IDfg', 'Name']
# )

In [ ]:
# # Drop Season Column
# players_off_output = players_off_output.drop(columns='Season')
# pd.set_option('display.float_format', '{:.2f}'.format)

# # Reorder columns to have 'avg_wOBA' and 'wOBA_2023' next to each other at the end
# column_order = [
#     col for col in players_off_output.columns if col not in ['avg_wOBA', 'wOBA_2023']
# ] + ['avg_wOBA', 'wOBA_2023']

# players_off_output = players_off_output[column_order]

In [24]:
avg_woba = players_batting_output['avg_woba']
woba_2023 = players_batting_output['woba_2023']

z_scores_avg_woba = zscore(avg_woba)
z_score_woba_2023 = zscore(woba_2023)

# Add new columns with Z-scores to the DataFrame
players_batting_output['z_scores_avg_woba'] = z_scores_avg_woba

/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_3726/4071663568.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_batting_output['z_scores_avg_woba'] = z_scores_avg_woba


In [26]:
# # Calculate z-scores for avg_wOBA and wOBA_2023
# players_off_output['zscore_avg_wOBA'] = z_scores_avg_woba 
# players_off_output['zscore_wOBA_2023'] = z_score_woba_2023

# Calculate the difference between z-scores
# players_off_output['zscore_difference'] = players_off_output['zscore_wOBA_2023'] - players_off_output['zscore_avg_wOBA']

players_batting_output['zscore_difference'] = z_score_woba_2023 - z_scores_avg_woba 

players_batting_output

/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_3726/1141941982.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_batting_output['zscore_difference'] = z_score_woba_2023 - z_scores_avg_woba


,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,est_woba,est_woba_minus_woba_diff,zscore_difference,avg_woba,woba_2023,z_scores_avg_woba
0,408234,2020.67,502.67,351.33,0.26,0.25,0.02,0.37,0.40,-0.03,0.31,-0.01,0.14,0.30,0.29,-0.94
1,444482,2020.00,516.25,365.50,0.27,0.25,0.02,0.45,0.40,0.05,0.32,0.02,-0.67,0.33,0.30,0.17
2,453568,2020.00,622.25,457.50,0.28,0.28,0.01,0.48,0.45,0.03,0.34,0.01,0.28,0.35,0.36,0.73
3,455117,2020.00,395.75,252.50,0.20,0.19,0.00,0.35,0.33,0.01,0.27,0.00,-0.31,0.27,0.24,-1.84
4,456781,2020.67,292.00,213.67,0.30,0.29,0.01,0.42,0.42,-0.01,0.33,-0.00,0.54,0.33,0.34,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,681082,2022.00,466.00,340.00,0.23,0.24,-0.00,0.36,0.34,0.02,0.28,0.01,1.58,0.29,0.34,-1.27
240,682928,2022.00,302.00,238.00,0.25,0.24,0.01,0.32,0.32,0.01,0.27,0.00,1.56,0.27,0.31,-1.98
241,682985,2022.00,418.00,259.00,0.25,0.25,0.01,0.36,0.39,-0.03,0.32,-0.01,1.86,0.30,0.36,-0.82
242,683734,2021.50,512.00,370.00,0.25,0.25,0.00,0.41,0.42,-0.01,0.33,-0.01,0.40,0.32,0.32,-0.38


In [11]:
#  #Write the DataFrame to a CSV file
# players_off_output.to_csv('Resources/off_output_for_learning.csv', index=False)

In [27]:
players_batting_output.to_csv('Resources/batting_for_learning.csv', index=False)

## Pitchers by zERA ZScore

In [13]:
from pybaseball import statcast_pitcher_expected_stats

# get data for all qualified pitchers in 2019
pitching_18 = statcast_pitcher_expected_stats(2018)
pitching_19 = statcast_pitcher_expected_stats(2019)
pitching_21 = statcast_pitcher_expected_stats(2021)
pitching_22 = statcast_pitcher_expected_stats(2022)
pitching_23 = statcast_pitcher_expected_stats(2023)

pitcher_combined_data = pd.concat([pitching_18, pitching_19, pitching_21, pitching_22, pitching_23])
pitcher_combined_data

,last_name,first_name,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,xera,era_minus_xera_diff
0,Keuchel,Dallas,572971,2018,874,661,0.263,0.247,0.016,0.393,0.362,0.031,0.305,0.293,0.012,3.74,3.60,0.138
1,Shields,James,448306,2018,871,630,0.245,0.262,-0.017,0.435,0.466,-0.031,0.326,0.350,-0.024,4.53,5.33,-0.801
2,Scherzer,Max,453286,2018,866,503,0.188,0.185,0.003,0.332,0.316,0.016,0.252,0.248,0.004,2.53,2.56,-0.031
3,Freeland,Kyle,607536,2018,844,595,0.240,0.238,0.002,0.358,0.373,-0.015,0.292,0.300,-0.008,2.85,3.79,-0.943
4,Kluber,Corey,446372,2018,842,583,0.223,0.227,-0.004,0.367,0.369,-0.002,0.270,0.277,-0.007,2.89,3.20,-0.312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,Leone,Dominic,608678,2023,160,104,0.243,0.245,-0.002,0.450,0.447,0.003,0.320,0.333,-0.013,4.23,4.69,-0.464
366,Vesia,Alex,681911,2023,159,92,0.282,0.230,0.052,0.437,0.363,0.074,0.344,0.296,0.048,5.45,3.63,1.822
367,Coulombe,Danny,543056,2023,159,100,0.230,0.205,0.025,0.324,0.310,0.014,0.270,0.255,0.015,2.77,2.66,0.109
368,Leclerc,José,600917,2023,158,97,0.190,0.176,0.014,0.307,0.306,0.001,0.266,0.261,0.005,2.82,2.79,0.027


In [14]:
relevant_years = [2018, 2019, 2021, 2022]
filtered_data = pitcher_combined_data[pitcher_combined_data['year'].isin(relevant_years)]

pitching_data_2023 = pitcher_combined_data[pitcher_combined_data['year'] == 2023]
# Merge the average wOBA with the 2023 wOBA for each player
grouped_data = filtered_data.groupby(['player_id']).mean().reset_index().rename(columns={'xera': 'avg_zERA'})

pitching_data_2023

/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_3726/2014980854.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_data = filtered_data.groupby(['player_id']).mean().reset_index().rename(columns={'xera': 'avg_zERA'})


,last_name,first_name,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,xera,era_minus_xera_diff
0,Alcantara,Sandy,645261,2023,675,488,0.246,0.263,-0.017,0.387,0.409,-0.022,0.296,0.317,-0.021,4.11,4.21,-0.102
1,Webb,Logan,657277,2023,670,477,0.248,0.250,-0.002,0.385,0.399,-0.014,0.289,0.297,-0.008,3.35,3.65,-0.295
2,Keller,Mitch,656605,2023,665,436,0.251,0.243,0.008,0.405,0.390,0.015,0.312,0.308,0.004,4.22,3.95,0.271
3,Mikolas,Miles,571945,2023,650,502,0.277,0.284,-0.007,0.421,0.465,-0.044,0.319,0.342,-0.023,4.55,4.98,-0.431
4,Cole,Gerrit,543037,2023,643,422,0.219,0.238,-0.019,0.343,0.392,-0.049,0.275,0.302,-0.027,3.03,3.79,-0.759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,Leone,Dominic,608678,2023,160,104,0.243,0.245,-0.002,0.450,0.447,0.003,0.320,0.333,-0.013,4.23,4.69,-0.464
366,Vesia,Alex,681911,2023,159,92,0.282,0.230,0.052,0.437,0.363,0.074,0.344,0.296,0.048,5.45,3.63,1.822
367,Coulombe,Danny,543056,2023,159,100,0.230,0.205,0.025,0.324,0.310,0.014,0.270,0.255,0.015,2.77,2.66,0.109
368,Leclerc,José,600917,2023,158,97,0.190,0.176,0.014,0.307,0.306,0.001,0.266,0.261,0.005,2.82,2.79,0.027


In [15]:
zera_2023 = pitching_data_2023[['player_id', 'xera']].rename(columns={'xera': 'zERA_2023'})
zera_2023

,player_id,zERA_2023
0,645261,4.21
1,657277,3.65
2,656605,3.95
3,571945,4.98
4,543037,3.79
...,...,...
365,608678,4.69
366,681911,3.63
367,543056,2.66
368,600917,2.79


In [16]:
# Merge player names with the result DataFrame
# Merge player names and calculated results
pitcher_output = pd.merge(
    grouped_data,
    zera_2023,
    on=['player_id']
)

# Display the result
pitcher_output

,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,avg_zERA,era_minus_xera_diff,zERA_2023
0,425794,2020.666667,792.00,571.333333,0.251333,0.262000,-0.010667,0.391333,0.412000,-0.020667,0.303000,0.319667,-0.016667,3.650000,4.396667,-0.744333,7.70
1,425844,2020.000000,732.75,550.250000,0.249250,0.258250,-0.009000,0.401250,0.416500,-0.015250,0.294750,0.309750,-0.015000,3.495000,4.100000,-0.605750,5.36
2,434378,2019.666667,782.00,480.666667,0.186000,0.192333,-0.006333,0.330333,0.328667,0.001667,0.241333,0.246667,-0.005333,2.283333,2.553333,-0.268333,3.55
3,445276,2020.000000,272.50,163.750000,0.189500,0.189000,0.000500,0.336250,0.313000,0.023250,0.261000,0.256500,0.004500,3.080000,2.762500,0.317500,2.71
4,446372,2020.333333,624.00,441.000000,0.248000,0.242667,0.005333,0.389000,0.385667,0.003333,0.298667,0.298667,0.000000,3.683333,3.743333,-0.060667,6.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,678394,2022.000000,268.00,184.000000,0.315000,0.246000,0.069000,0.416000,0.347000,0.069000,0.355000,0.303000,0.052000,4.710000,3.800000,0.909000,4.09
269,680686,2021.500000,478.00,308.000000,0.238500,0.219000,0.019500,0.497500,0.436500,0.061000,0.349000,0.322000,0.027000,5.255000,4.380000,0.870500,4.72
270,680694,2022.000000,509.00,344.000000,0.265000,0.253000,0.012000,0.419000,0.404000,0.015000,0.334000,0.327000,0.007000,4.890000,4.490000,0.405000,4.01
271,681911,2022.000000,227.00,122.000000,0.187000,0.188000,-0.001000,0.247000,0.295000,-0.048000,0.244000,0.261000,-0.017000,2.150000,2.790000,-0.637000,3.63


In [17]:
pitcher_output = pitcher_output.drop(columns='year')
pd.set_option('display.float_format', '{:.2f}'.format)

In [18]:
avg_zera = pitcher_output['avg_zERA']
zera_2023 = pitcher_output['zERA_2023']
# 
z_scores_avg_zera = zscore(avg_zera)
z_score_zera_2023 = zscore(zera_2023)

In [19]:
pitcher_output['zscore_difference'] = z_score_zera_2023 - z_scores_avg_zera
# Display the result

column_order = [
    'player_id', 'pa', 'bip', 'ba', 'est_ba', 'est_ba_minus_ba_diff',
    'slg', 'est_slg', 'est_slg_minus_slg_diff', 'woba', 'est_woba',
    'est_woba_minus_woba_diff', 'era', 'era_minus_xera_diff', 'avg_zERA',
    'zERA_2023', 'zscore_difference'
]

# Reorder the columns
pitcher_output = pitcher_output[column_order]

# Display the result
pitcher_output

,player_id,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,era_minus_xera_diff,avg_zERA,zERA_2023,zscore_difference
0,425794,792.00,571.33,0.25,0.26,-0.01,0.39,0.41,-0.02,0.30,0.32,-0.02,3.65,-0.74,4.40,7.70,2.83
1,425844,732.75,550.25,0.25,0.26,-0.01,0.40,0.42,-0.02,0.29,0.31,-0.01,3.50,-0.61,4.10,5.36,0.89
2,434378,782.00,480.67,0.19,0.19,-0.01,0.33,0.33,0.00,0.24,0.25,-0.01,2.28,-0.27,2.55,3.55,1.20
3,445276,272.50,163.75,0.19,0.19,0.00,0.34,0.31,0.02,0.26,0.26,0.00,3.08,0.32,2.76,2.71,0.06
4,446372,624.00,441.00,0.25,0.24,0.01,0.39,0.39,0.00,0.30,0.30,0.00,3.68,-0.06,3.74,6.18,2.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,678394,268.00,184.00,0.32,0.25,0.07,0.42,0.35,0.07,0.35,0.30,0.05,4.71,0.91,3.80,4.09,0.02
269,680686,478.00,308.00,0.24,0.22,0.02,0.50,0.44,0.06,0.35,0.32,0.03,5.26,0.87,4.38,4.72,-0.14
270,680694,509.00,344.00,0.27,0.25,0.01,0.42,0.40,0.01,0.33,0.33,0.01,4.89,0.41,4.49,4.01,-1.00
271,681911,227.00,122.00,0.19,0.19,-0.00,0.25,0.29,-0.05,0.24,0.26,-0.02,2.15,-0.64,2.79,3.63,0.95


In [20]:
 #Write the DataFrame to a CSV file
pitcher_output.to_csv('Resources/pitcher_output_for_learning.csv', index=False)